In [2]:
import os
import json
import pandas
import functools

In [12]:
files = filter(lambda a: a.endswith(".json"), os.listdir("results/"))

doctor_lists = []

for fnm in files:
    with open(os.path.join("results", fnm), "r") as f:
        doctor_lists.append(json.load(f))

def extend(L1, L2):
    L1.extend(L2)
    return L1
        
all_doctors = functools.reduce(extend, doctor_lists)

In [13]:
len(all_doctors)

6119

In [14]:
all_doctors[0]

{'accepting_new_patients': 'No',
 'addresses': ['2791 First Ave\nTerrace, BC\nV8G 0G2'],
 'gender': 'Male',
 'name': 'Appleton, Geoffrey Marsden',
 'practice': 'General Family Practice'}

In [19]:
names = [d["name"] for d in all_doctors]
addr = [d["addresses"][0] for d in all_doctors]
gender = [d["gender"] for d in all_doctors]
practice = [d["practice"] for d in all_doctors]
newpat = [d["accepting_new_patients"] for d in all_doctors]

gfp = ["General Family Practice" in p for p in practice]
ccfp = ["CCFP" in p for p in practice]
ccfp_em = ["CCFP (EM)" in p for p in practice]
emer_med = ["Emergency Medicine+" in p for p in practice]
an_path = ["RCPSC - Anatomical Pathology" in p for p in practice]
int_med = ["RCPSC - Internal Medicine" in p for p in practice]
med_bio = ["RCPSC - Medical Biochemistry" in p for p in practice]
gen_surg = ["RCPSC - General Surgery" in p for p in practice]

In [23]:
data = pandas.DataFrame(
    {
        "Name": names,
        "Address": addr,
        "Gender": gender,
        "Accepting_New_Patients": newpat,
        "General_Family_Practice": gfp,
        "CCFP": ccfp,
        "CCFP_EM": ccfp_em,
        "Emergency_Medicine": emer_med,
        "Anatomical_Pathology": an_path,
        "Internal_Medicine": int_med,
        "Medical_Biochemistry": med_bio,
        "General_Surgery": gen_surg
    })

In [24]:
data.to_csv("CPSBC-data-v0.1.0.csv")